# LSTM con embeddings UBA

Modelo básico con los embeddings de fasttext

In [1]:
import pandas as pd
import csv
import numpy as np
import tensorflow as tf
import random

np.random.seed(2019)
tf.random.set_random_seed(2019)
random.seed(2019)

df_dev = pd.read_table("../../../data/es/dev_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_train = pd.read_table("../../../data/es/train_es.tsv", index_col="id", quoting=csv.QUOTE_NONE)
df_test = pd.read_table("../../../data/es/reference_es.tsv", header=None, 
                        names=["text", "HS", "TR", "AG"], quoting=csv.QUOTE_NONE)


text_train, y_train = df_train["text"], df_train["HS"]
text_dev, y_dev = df_dev["text"], df_dev["HS"]
text_test, y_test = df_test["text"], df_test["HS"]

print("Instancias de entrenamiento: {}".format(len(df_train)))
print("Instancias de desarrollo: {}".format(len(df_dev)))


Instancias de entrenamiento: 4500
Instancias de desarrollo: 500


Tengo que hacer dos cosas:

- Primero, convertir los tweets a secuencias de texto
- Luego, paddear las secuencias a cierta longitud (Keras necesita esto para poder paralelizar cálculo)

In [2]:
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

num_words = 200000

tokenizer = Tokenizer(num_words=num_words)
tokenizer.fit_on_texts(text_train)

X_train = tokenizer.texts_to_sequences(text_train)
X_dev = tokenizer.texts_to_sequences(text_dev)
X_test = tokenizer.texts_to_sequences(text_test)
max_length = 30

X_train = pad_sequences(X_train, max_length)
X_dev = pad_sequences(X_dev, max_length)
X_test = pad_sequences(X_test, max_length)

Using TensorFlow backend.


Carguemos embeddings

In [3]:
import os
from glob import glob

path_to_embeddings = os.path.expanduser("/home/jmperez/WordVectors/")

print("Available embeddings: ", glob(os.path.join(path_to_embeddings, "*.vec")))

Available embeddings:  ['/home/jmperez/WordVectors/UBA_w5_200.vec', '/home/jmperez/WordVectors/wiki.es.vec', '/home/jmperez/WordVectors/UBA_w5_300.vec']


# Twitter Embeddings

In [4]:
import numpy as np

word_to_vec = {}

with open(os.path.join(path_to_embeddings, "UBA_w5_300.vec")) as f:
    for line in f:
        values = line.split()
        word = values[0]
        try:
            vec = np.asarray(values[1:], dtype="float32")
        except:
            print(("*" * 80  + "\n")*3)
            print("Problema con la sig línea:")
            print(values[:10])
            word = values[1]
            vec = np.asarray(values[2:], dtype="float32")
        word_to_vec[word] = vec
        
embedding_size = len(word_to_vec["hola"])

********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '.', '-0.22232', '0.0052569', '0.47066', '0.13836', '0.15991', '0.19504', '0.00067885', '0.020299']
********************************************************************************
********************************************************************************
********************************************************************************

Problema con la sig línea:
['.', '...', '-0.11666', '-0.083768', '0.028919', '0.29973', '0.21017', '0.27808', '0.063251', '0.090223']
********************************************************************************
********************************************************************************
********************************************************************************

P

In [5]:
# create a weight matrix for words in training docs
embedding_matrix = np.zeros((num_words, embedding_size))
for word, i in tokenizer.word_index.items():
    embedding_vector = word_to_vec.get(word)
    if embedding_vector is not None:
        embedding_matrix[i] = embedding_vector

In [6]:
embedding_matrix.shape

(200000, 300)

In [7]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import CuDNNLSTM, Dropout
from keras.optimizers import Adam
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence


model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(CuDNNLSTM(100))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
optimizer_args = {
    "lr": 0.001,
    "decay": 0.01
}
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_1 (Embedding)      (None, 30, 300)           60000000  
_________________________________________________________________
cu_dnnlstm_1 (CuDNNLSTM)     (None, 100)               160800    
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 128)               12928     
_________________________________________________________________
dropout_2 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 129       
Total params: 60,173,857
Trainable params: 173,857
Non-trainable params: 60,000,000
__________________________________________________________

In [8]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 115us/step
Loss           : 0.5202
Accuracy       : 0.7620
Precision(1)   : 0.7249
Precision(1)   : 0.7934
Precision(avg) : 0.7591

Recall(1)      : 0.7477
Recall(0)      : 0.7734
Recall(avg)    : 0.7606

F1(1)          : 0.7361
F1(0)          : 0.7832
F1(avg)        : 0.7597


Evaluación sobre test
1600/1600 [==============================] - 0s 105us/step
Loss           : 0.6128
Accuracy       : 0.6969
Precision(1)   : 0.6178
Precision(1)   : 0.7655
Precision(avg) : 0.6916

Recall(1)      : 0.6955
Recall(0)      : 0.6979
Recall(avg)    : 0.6967

F1(1)          : 0.6543
F1(0)          : 0.7301
F1(avg)        : 0.6922


## Bidirectional LSTM

In [9]:
from keras.layers import Bidirectional
from keras.layers.embeddings import Embedding
from keras.preprocessing import sequence

embedding_vector_length = 32

model = Sequential()
model.add(Embedding(num_words, embedding_size, input_length=max_length, 
                    weights=[embedding_matrix], trainable=False))
model.add(Bidirectional(CuDNNLSTM(100)))
model.add(Dropout(0.5))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(1, activation='sigmoid'))
model.compile(loss='binary_crossentropy', 
              optimizer=Adam(**optimizer_args), 
              metrics=['accuracy'])
print(model.summary())

model.fit(X_train, y_train, validation_data=(X_dev, y_dev), epochs=10, batch_size=32)


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      (None, 30, 300)           60000000  
_________________________________________________________________
bidirectional_1 (Bidirection (None, 200)               321600    
_________________________________________________________________
dropout_3 (Dropout)          (None, 200)               0         
_________________________________________________________________
dense_3 (Dense)              (None, 128)               25728     
_________________________________________________________________
dropout_4 (Dropout)          (None, 128)               0         
_________________________________________________________________
dense_4 (Dense)              (None, 1)                 129       
Total params: 60,347,457
Trainable params: 347,457
Non-trainable params: 60,000,000
__________________________________________________________

In [10]:
from hate.utils import print_evaluation

print("Evaluación sobre dev")
print_evaluation(model, X_dev, y_dev)
print("\n\nEvaluación sobre test")
print_evaluation(model, X_test, y_test)



Evaluación sobre dev
500/500 [==============================] - 0s 132us/step
Loss           : 0.5171
Accuracy       : 0.7700
Precision(1)   : 0.7277
Precision(1)   : 0.8075
Precision(avg) : 0.7676

Recall(1)      : 0.7703
Recall(0)      : 0.7698
Recall(avg)    : 0.7700

F1(1)          : 0.7484
F1(0)          : 0.7882
F1(avg)        : 0.7683


Evaluación sobre test
1600/1600 [==============================] - 0s 135us/step
Loss           : 0.6145
Accuracy       : 0.6787
Precision(1)   : 0.5892
Precision(1)   : 0.7724
Precision(avg) : 0.6808

Recall(1)      : 0.7303
Recall(0)      : 0.6426
Recall(avg)    : 0.6864

F1(1)          : 0.6522
F1(0)          : 0.7015
F1(avg)        : 0.6769
